<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part2_Chapter06_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EDA

In [10]:
train.shape, test.shape

((25519, 19), (7591, 18))

In [12]:
train.isnull().sum() #OCCUPATION_TYPE
test.isnull().sum() #X

,0
ID,0
CODE_GENDER,0
FLAG_OWN_CAR,0
FLAG_OWN_REALTY,0
CNT_CHILDREN,0
AMT_INCOME_TOTAL,0
NAME_INCOME_TYPE,0
NAME_EDUCATION_TYPE,0
NAME_FAMILY_STATUS,0
NAME_HOUSING_TYPE,0


In [3]:
display(train.head(3))
display(test.head(3))

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,STATUS
0,5010092,F,N,N,0,225000.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-10304,-1682,1,0,0,0,Laborers,2.0,0
1,5116701,M,N,Y,1,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-10189,-1189,1,1,1,0,NaN,3.0,0
2,5069290,F,N,Y,0,270000.0,Commercial associate,Incomplete higher,Married,House / apartment,-14783,-3397,1,0,0,0,Core staff,2.0,0


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5037176,F,N,Y,1,337500.0,Commercial associate,Higher education,Single / not married,House / apartment,-14408,-431,1,0,0,0,Accountants,2.0
1,5142150,M,Y,Y,0,450000.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15960,-3574,1,0,0,0,High skill tech staff,1.0
2,5094889,F,N,Y,0,90000.0,Working,Higher education,Single / not married,House / apartment,-21532,-4630,1,0,0,0,Laborers,1.0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25519 entries, 0 to 25518
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   25519 non-null  int64  
 1   CODE_GENDER          25519 non-null  object 
 2   FLAG_OWN_CAR         25519 non-null  object 
 3   FLAG_OWN_REALTY      25519 non-null  object 
 4   CNT_CHILDREN         25519 non-null  int64  
 5   AMT_INCOME_TOTAL     25519 non-null  float64
 6   NAME_INCOME_TYPE     25519 non-null  object 
 7   NAME_EDUCATION_TYPE  25519 non-null  object 
 8   NAME_FAMILY_STATUS   25519 non-null  object 
 9   NAME_HOUSING_TYPE    25519 non-null  object 
 10  DAYS_BIRTH           25519 non-null  int64  
 11  DAYS_EMPLOYED        25519 non-null  int64  
 12  FLAG_MOBIL           25519 non-null  int64  
 13  FLAG_WORK_PHONE      25519 non-null  int64  
 14  FLAG_PHONE           25519 non-null  int64  
 15  FLAG_EMAIL           25519 non-null 

#0. 데이터 불러오기

In [81]:
import pandas as pd
train = pd.read_csv('creditcard_train.csv')
test = pd.read_csv('creditcard_test.csv')

#데이터 전처리

In [82]:
#결측치 처리
train.dropna(subset=['OCCUPATION_TYPE'], axis=0, inplace=True)

#ID 는 ID -> 예측 정상화됨
#FLAG_MOBIL 은 전체다 1 -> 삭제
train.drop(['ID', 'FLAG_MOBIL'], axis=1, inplace=True)
test.drop(['ID', 'FLAG_MOBIL'], axis=1, inplace=True)

In [83]:
#타겟 변수 분리
target = train.pop('STATUS')

In [84]:
data = pd.concat([train, test], axis=0)
print("train shape : ", train.shape)
print("test shape : ", test.shape)
print("concat shape : ", data.shape)

#object -> LE
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cols = data.select_dtypes(include='object').columns
for col in cols:
  data[col] = le.fit_transform(data[col])


#num -> Robust
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data = scaler.fit_transform(data)


#데이터 분할
train = data[:len(train), :]
test = data[len(train):, :]

print(train.shape, test.shape)

train shape :  (17543, 16)
test shape :  (7591, 16)
concat shape :  (25134, 16)
(17543, 16) (7591, 16)


#모델

In [95]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.3, random_state = 99)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(12280, 16) (5263, 16) (12280,) (5263,)


In [96]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(train, target)

from sklearn.metrics import f1_score
y_pred = rfc.predict(X_val)
score = f1_score(y_val, y_pred)
print('f1_score : ', score)

f1_score :  0.5771812080536913
